# Compute Marginal using pandas

In [7]:
%pylab inline
import itertools
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [8]:
tuples = [(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]
mindex = pd.MultiIndex.from_tuples(tuples, names=['a', 'b'])
factor1 = pd.Series([0.5,0.8,0.1,0.0,0.3,0.7], index=mindex)

tuples = [(1,1),(1,2),(2,1),(2,2)]
mindex = pd.MultiIndex.from_tuples(tuples, names=['b', 'c'])
factor2 = pd.Series([0.5,0.7,0.1,0.2], index=mindex)

tuples = list(itertools.product([1,2,3],[1,2],[1,2,3]))
mindex = pd.MultiIndex.from_tuples(tuples, names=['a', 'c', 'd'])
factor3 = pd.Series([
    0.5,0.7,0.1,0.2,0.3,0.2,
    0.4,0.3,0.8,0.3,0.9,0.0,
    0.2,0.0,0.1,0.4,0.0,0.2 ], index=mindex)

In [5]:
def FactorProduct(factor1, factor2):
    allV =list(set(factor1.index.names) | set(factor2.index.names))
    commonV = list(set(factor1.index.names) & set(factor2.index.names))
    merged = pd.merge(factor1.reset_index(), factor2.reset_index(), on=commonV)
    merged.set_index(allV, inplace=True)
    return merged["0_x"] * merged["0_y"]

def ObserveEvidence(factor, **evidence):
    observed = factor.copy()
    for variableName, value in evidence.iteritems():
        resetV = observed.reset_index(level=variableName)
        resetV.loc[resetV[variableName] != value, 0] = 0
        observed = resetV.set_index(variableName, append=True)[0]
    return observed

def Marginalize(factor, variable):
    return factor.unstack(variable).sum(axis=1)

def JointDistribution(factors):
    return reduce(FactorProduct, factors, factors[0])

**I guess that should be self-explanatory by now:**

In [35]:
def ComputeMarginal(variables, factors, **evidence):
    data = JointDistribution(factors)
    data = ObserveEvidence(data, **evidence)
    vs = sorted(list(set(data.index.names) - set(variables)))
    data = Marginalize(data, vs)
    data = data.astype(float) / data.sum()
    return data

**Here is how it is used**

In [36]:
ComputeMarginal(['a'], [factor1, factor2, factor3], c=2)

a
1    0.668874
2    0.026490
3    0.304636
dtype: float64

In [37]:
ComputeMarginal(['b','d'], [factor1, factor2, factor3], c=1, a=1)

b  d
1  1    0.254375
   2    0.356125
   3    0.050875
2  1    0.130240
   2    0.182336
   3    0.026048
dtype: float64